In [169]:
### python 3.6

### to do:
#1 load data
#2 clean data
#3 train
#4 evaluate
#5 feature selection & reevaluation

In [172]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import itertools


In [173]:
#1 load data
# read CSV file
transactions_raw = pd.read_csv('Transaction_Classification.csv', delimiter = ";")

In [174]:
#2 clean data
# get some overview
transactions_raw.info()
# some null values exist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         209 non-null    int64  
 1   Auftragskonto                      168 non-null    float64
 2   Buchungstag                        209 non-null    object 
 3   Valutadatum                        209 non-null    object 
 4   Buchungstext                       209 non-null    object 
 5   Verwendungszweck                   209 non-null    object 
 6   Beguenstigter/Zahlungspflichtiger  209 non-null    object 
 7   Kontonummer                        208 non-null    object 
 8   BLZ                                208 non-null    object 
 9   Betrag                             209 non-null    object 
 10  Waehrung                           209 non-null    object 
 11  label                              209 non-null    object 

In [175]:
transactions_raw.head()
# todo: check wheather waehrung always is EUR, is buchungstag always equal to valutadatum?

,Unnamed: 0,Auftragskonto,Buchungstag,Valutadatum,Buchungstext,Verwendungszweck,Beguenstigter/Zahlungspflichtiger,Kontonummer,BLZ,Betrag,Waehrung,label
0,0,89990201.0,28.07.2016,28.07.2016,Lohn / Gehalt,Gehalt Adorsys GmbH & Co. KG End-To-End-Ref.: ...,Adorsys GmbH & Co. KG,7807800780,25190001,2000.00,EUR,income
1,1,89990201.0,27.07.2016,27.07.2016,Miete,Byladem1Sbt De12773501123456789889 Miete Beuth...,Georg Tasche,DE31251900019123456780,VOHADE2HXXX,-670.00,EUR,living
2,2,89990201.0,21.07.2016,21.07.2016,Bargeld,"21.07/16.34Uhr Nuernberg All Eur 70,00 Geb.Eur...",Bargeld,9999900780,25190001,-70.00,EUR,private
3,3,89990201.0,20.07.2016,20.07.2016,Lebensmittel / Getraenke,2831 Edeka Neubauer Nuernb.//Nuernb 2016-07-20...,Kartenzahlung,9736000780,25190001,-73.21,EUR,standardOfLiving
4,4,89990201.0,18.07.2016,18.07.2016,Spontanausgabe,Amazon,neue Playstation,9988776655,25125100,-363,EUR,leisure


In [176]:
# look at distribution of classes
transactions_raw["label"].value_counts()
# class imbalance

leisure             65
standardOfLiving    47
finance             33
living              26
private             21
income              17
Name: label, dtype: int64

In [177]:
# look at unique values ofattributes
for columnName, columnData in transactions_raw.iteritems():
    print(columnName)
    print(columnData.unique())

Unnamed: 0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208]
Auftragskonto
[89990201. 89990210.       nan]
Buchungstag
['28.07.2016' '27.07.2016' '21.07.2016' '20.07.2016' '18.07.2016'
 '15.07.2016' '0

In [178]:
# remove währung as there is only one value
# remove original id
transactions = transactions_raw
transactions = transactions.drop(["Waehrung"], axis=1)
transactions = transactions.drop(["Unnamed: 0"], axis=1)

In [179]:
# set nan to "missing" for Auftragskonto to be usable for classifier
transactions['Auftragskonto'].fillna('missing', inplace=True)

# delete two rows with missing values, maybe leave later in case Kontonummer or BLZ will be removed completely
transactions = transactions.dropna()
transactions.reset_index(drop=True)
transactions.isnull().sum()
# all missing values dealt with

Auftragskonto                        0
Buchungstag                          0
Valutadatum                          0
Buchungstext                         0
Verwendungszweck                     0
Beguenstigter/Zahlungspflichtiger    0
Kontonummer                          0
BLZ                                  0
Betrag                               0
label                                0
dtype: int64

In [180]:
# check if Valuta and Buchungsdatum are always equal
buchungstag = transactions[['Buchungstag']]
buchungstag.reset_index(drop=True, inplace=True)
valutadatum = transactions[['Valutadatum']]
valutadatum.reset_index(drop=True, inplace=True)

for i in range (buchungstag.size):
    if buchungstag.loc[i, "Buchungstag"] != valutadatum.loc[i, "Valutadatum"]:
        print("difference:", i,buchungstag.loc[i, "Buchungstag"], valutadatum.loc[i, "Valutadatum"])
# since they only differ in one of over 200 cases, one of them can be disregarded quite safely
transactions= transactions.drop(["Valutadatum"], axis=1)



difference: 124 11.02.2016 12.02.2016


In [168]:

model = GaussianNB()
classes = transactions[["label"]].values.ravel()
features_all = ['Auftragskonto', 'Buchungstag','Buchungstext', 'Verwendungszweck', 'Beguenstigter/Zahlungspflichtiger','Kontonummer', 'BLZ', 'Betrag']
features_selection = []

for i in range(1, len(features_all)+1):
    combinations = list(itertools.combinations(features_all, i))
    max_score = 0
    for j in range(len(combinations)):
        features_selection = list(combinations[j])
        counts = pd.get_dummies(transactions[features_selection])
        scores = cross_val_score(model, counts, classes, cv=17, scoring ="accuracy")  #cv = 50 in summary
        mean_score = scores.mean()
        #print(features_selection)
        #print(j, mean_score)
        #print("j",j)
        if mean_score > max_score:
             max_score = mean_score
             best_feature_selection = features_selection
    print("max score for",i, "attribute(s) is:", max_score)
    print("features selected:" ,best_feature_selection)
    print()

if i == len(features_all):
    print("all feature combinations evaluated")
    


max score for 1 attribute(s) is: 0.8642533936651584
features selected: ['Beguenstigter/Zahlungspflichtiger']

max score for 2 attribute(s) is: 0.9321266968325792
features selected: ['Beguenstigter/Zahlungspflichtiger', 'BLZ']

max score for 3 attribute(s) is: 0.941553544494721
features selected: ['Beguenstigter/Zahlungspflichtiger', 'BLZ', 'Betrag']

max score for 4 attribute(s) is: 0.9513574660633485
features selected: ['Auftragskonto', 'Beguenstigter/Zahlungspflichtiger', 'BLZ', 'Betrag']

max score for 5 attribute(s) is: 0.9513574660633485
features selected: ['Auftragskonto', 'Buchungstext', 'Beguenstigter/Zahlungspflichtiger', 'BLZ', 'Betrag']

max score for 6 attribute(s) is: 0.9513574660633485
features selected: ['Auftragskonto', 'Buchungstext', 'Verwendungszweck', 'Beguenstigter/Zahlungspflichtiger', 'BLZ', 'Betrag']

max score for 7 attribute(s) is: 0.9513574660633485
features selected: ['Auftragskonto', 'Buchungstext', 'Verwendungszweck', 'Beguenstigter/Zahlungspflichtiger', '

In [188]:
#3 train and #4 evaluate iteratively during cross validation
# set model type
model = GaussianNB()
# selecting features, choose from ['Auftragskonto', 'Buchungstag','Buchungstext', 'Verwendungszweck', 'Beguenstigter/Zahlungspflichtiger','Kontonummer', 'BLZ', 'Betrag']
attr =['Beguenstigter/Zahlungspflichtiger', 'BLZ']


# training data and classes
counts = pd.get_dummies(transactions[attr])
classes = transactions[["label"]].values.ravel()

scores = cross_val_score(model, counts, classes, cv=17, scoring ="accuracy")

# Print the accuracy of each fold:
# print(scores) test

# Print the mean accuracy of all 5 folds
print("mean accuracy:", scores.mean())

mean accuracy: 0.9321266968325792
